In [1]:
import numpy as np
import musdb
import IPython.display as ipd
import openunmix as opmux
import torch
import h5py

from tqdm.autonotebook import tqdm

# Setup

In [2]:
musdb_path = "/home/paco/TFM/data/MUSDB18/"
data_path = "/home/paco/TFM/data/"

In [3]:
np.random.seed(42)
torch.manual_seed(42)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

In [5]:
print("Num GPUs Available: ", torch.cuda.get_device_name())
print("GPU available:", torch.cuda.is_available())

Num GPUs Available:  GeForce RTX 3060 Laptop GPU
GPU available: True


# Definimos los generadores de datos...

In [6]:
def h5_generator(data, source, target, n_samples=2**13, chunk_size=2**9, batch_size=8, randomize=True):
    # Cogemos n_samples elementos contiguos del dataset, empezando en algún punto al azar
    if(randomize):
        starting_point = len(data[source])-n_samples
        starting_point= 1 if(starting_point < 1) else starting_point
        starting_point = np.random.choice(starting_point)
    else:
        starting_point = 0
    data_source = data[source]
    data_target = data[target]
    
    # Si estamos pidiendo más muestras de las que hay, las devolvemos todas
    if((starting_point+n_samples) > data_source.shape[0]):
        n_samples = data_source.shape[0] - starting_point
    
    # Leemos los datos de un chunk
    for chunk_start in np.arange(starting_point, starting_point+n_samples, chunk_size):
        chunk_source = data_source[chunk_start:(chunk_start+chunk_size)]
        chunk_target = data_target[chunk_start:(chunk_start+chunk_size)]
        # Shuffle de source y target
        random_order = np.random.permutation(len(chunk_source))
        chunk_source = chunk_source[random_order]
        chunk_target = chunk_target[random_order]
        
        for jj in np.arange(0,len(chunk_source), 8):
            # Generamos predictor y target (audio original y la sección del target) y los cargamos como tensores de pytorch en "device" (gpu normalmente)
            x = chunk_source[jj:jj+8]
            y = chunk_target[jj:jj+8]
            # Devolvemos x e y
            yield x, y

In [7]:
class musdb_dataset(torch.utils.data.Dataset):
    def __init__(self,data,target='vocals',sequence_length=6):
        self.sequence_length = sequence_length
        self.target = target
        self.data = data

    def __getitem__(self, index):
        # Cogemos una canción al azar (en principio, si no contemplamos reemplazamiento, el batch size está limitado por el número de canciones en el dataset)
        track = self.data[index]
        # Generamos un trozo al azar y recortamos
        track.chunk_start = np.random.uniform(0, (track.duration - self.sequence_length))
        track.chunk_duration = self.sequence_length
        # Generamos predictor y target (audio original y la sección del target) y los cargamos como tensores de pytorch en "device" (gpu normalmente)
        x = torch.tensor(track.audio.T, dtype=torch.float32, device=device)
        y = torch.tensor(track.targets[self.target].audio.T, dtype=torch.float32, device=device)
        
        # Devolvemos x e y
        return x, y 


    def __len__(self):
        return len(self.data)

## Definimos algunos parámetros 

In [8]:
batch_size = 256
subtrack_length = 3
source = 'source'
targets = ['vocals', 'drums', 'bass', 'other']
target = targets[3]
# 8192 de training
train_samples = 2**13
# Todas las muestras de validación (1227)
val_samples = 2**11

In [9]:
train_file = h5py.File(data_path+'train_shuffled.h5', 'r')
val_file = h5py.File(data_path+'val_shuffled.h5', 'r')
test_file = h5py.File(data_path+'test_shuffled.h5', 'r')

# Definimos la arquitectura de la red...

### Primero, la transformada de Fourier para codificar/decodificar

In [10]:
sftf_window_size = 4096
stft_window_hop = 1024
stft_center = True

In [11]:
class RecSepSTFT_3(torch.nn.Module): # Buenos resultados con lr inicial = 1e-2
    def __init__(self):
        super().__init__()
        # Usamos la norma del complejo como hacen en OpenUnmix
        self.complex_norm = opmux.transforms.ComplexNorm()
        
        self.encoder_stft = opmux.transforms.TorchSTFT(n_fft=sftf_window_size, n_hop=stft_window_hop, center=stft_center)
        self.decoder_stft = opmux.transforms.TorchISTFT(n_fft=sftf_window_size, n_hop=stft_window_hop, center=stft_center)
        
        self.conv_init = torch.nn.Conv1d(in_channels = ((int(sftf_window_size/2)+1)*2), 
                                         out_channels =1024, 
                                         kernel_size=1,
                                         groups=1,padding_mode='circular',)
        self.bn_conv_init = torch.nn.BatchNorm1d(1024)
        
        self.conv_final = torch.nn.Conv1d(in_channels = 1024, 
                                         out_channels = ((int(sftf_window_size/2)+1)*2), 
                                         kernel_size=1,
                                         groups=1,padding_mode='circular',)
        self.bn_conv_final = torch.nn.BatchNorm1d((int(sftf_window_size/2)+1)*2)
        
        
        
        
#         self.fc1 = torch.nn.Linear(((int(sftf_window_size/2)+1)*2), 256)
        self.fc1 = torch.nn.Linear(1024, 256)
        self.bn1 = torch.nn.BatchNorm1d(256)
        
        
        self.lstm = torch.nn.LSTM(input_size=256,
                                  hidden_size=int(256/2),
                                  num_layers=3, 
                                  batch_first=True, 
                                  dropout=0, 
                                  bidirectional=True,
                                  proj_size=0)
        #self.fc_lstm_res = torch.nn.Linear(256*2,1024)
        self.dropout = torch.nn.Dropout(0)
        
        self.fc2 = torch.nn.Linear(256, 256)
        self.bn2 = torch.nn.BatchNorm1d(256)
        
        self.fc3 = torch.nn.Linear(256,256)
        self.bn3 = torch.nn.BatchNorm1d(256)
        
        self.fc4 = torch.nn.Linear(256,1024)
        self.bn4 = torch.nn.BatchNorm1d(1024)
#         self.fc4 = torch.nn.Linear(256,(int(sftf_window_size/2)+1)*2)
#         self.bn4 = torch.nn.BatchNorm1d((int(sftf_window_size/2)+1)*2)

    def forward(self, x):
        pre_mix = x
        # Calculamos la norma compleja (pasamos a dominio real)
        x = self.complex_norm(x)
        
        # Tenemos una entrada en formato: (batch, canal, feature/banda, secuencia)
        b_size, n_channel, n_feat, seq_len = x.size()
        
        # Pasamos la primera convolución
        x = torch.reshape(x, (b_size, n_channel*n_feat, seq_len))
        
        x = self.conv_init(x)
        x = self.bn_conv_init(x)
        x = torch.nn.functional.elu(x)
        x = torch.permute(x, (0,2,1))
        

        # Vamos a hacer un primer paso de codificación, para ello tenemos que dejar los datos en forma (batch, secuencia, features)
        # Permutamos los datos a formato (batch, secuencia, canal, banda)
#         x = torch.permute(x, (0,3,1,2))
        # Pasamos las dos últimas dimensiones (ahora son canal, banda) a una única ("desenrollamos" las features de cada canal en uno solo)
#         x = torch.reshape(x, (b_size, seq_len, n_channel * n_feat))
        # Ponemos una capa fully connected, batch norm, y activación
        x = self.fc1(x)
        # Para el batch norm hay que tener el tensor en formato (batch, features, sequence)
        x = torch.permute(x, (0,2,1))
        # Batch norm
        x = self.bn1(x)
        # Ahora tenemos los datos en formato  (batch, features, sequence)
        # La lstm los necesita en formato (batch, sequence, features)
        x = torch.permute(x, (0,2,1))
        # Activación
        x = torch.nn.functional.elu(x)
        x_skip_1 = x
        
        x = self.dropout(x)
        
        # Nos quedamos con los estados de cada step de la secuencia
        x_skip_lstm = x
        x=self.lstm(x)[0]
        x = x_skip_lstm + x
        x = torch.nn.functional.elu(x)
        
        # Ahora tenemos los datos en formato (batch, sequence, features)
        # Aplicamos fc, bn, activation de nuevo
        x = self.fc2(x)
        x = torch.permute(x, (0,2,1))
        x = self.bn2(x)
        x = torch.permute(x, (0,2,1))
        x = torch.nn.functional.elu(x)
        x_skip_2 = x
        
        # Aplicamos una capa fc más para conseguir que features tenga un tamaño compatible con (canales, bandas_fourier_iniciales)
        x = self.fc3(x)
        x = torch.permute(x, (0,2,1))
        x = self.bn3(x)
        x = torch.permute(x, (0,2,1))
        x = torch.nn.functional.elu(x)
        
        x = x + x_skip_1
        x = x + x_skip_2
        
        x = self.fc4(x)
        # Pasamos a formato (batch, features, secuencia)
        x = torch.permute(x, (0,2,1))
        x = self.bn4(x)
        x = torch.nn.functional.elu(x)
        # Aplicamos la convolución final
        x = self.conv_final(x)
        x = self.bn_conv_final(x)
        x = torch.nn.functional.elu(x)
        # Devolvemos a formato (batch, secuencia, features)
        x = torch.permute(x, (0,2,1))
        
        
        
        # Redistribuimos y giramos para dejar los datos en formato: (batch, canal, feature/banda, secuencia, complejo)
        x = torch.reshape(x, (b_size, seq_len, n_channel, n_feat, 1))
        x = torch.permute(x, (0,2,3,1,4))
        
        # Aplicamos una sigmoidal para obtener una soft mask, y la aplicamos a la entrada
        # x = torch.sigmoid(x)
        
        # Aplicamos x como una máscara a la stft de la entrada sin procesar
        x = pre_mix * x
        
        return x

In [12]:
# %%time
# Número de épocas a entrenar
n_epochs = 100
init_lr = 1e-2
min_lr = 1e-7
lr_factor = 1e-1

val_loss_counter = 0

train_samples = train_samples if (train_samples < len(train_file[source])) else len(train_file[source])
val_samples = val_samples if (val_samples < len(val_file[source])) else len(val_file[source])

progress_bar = tqdm(np.arange(n_epochs * train_samples))

# Instanciamos el modelo
model = RecSepSTFT_3()

# Cargamos el modelo a GPU/dispositivo, junto con las transformadas
model.to(device)

# Definimos el optimizador y la función de loss
optimizer = torch.optim.RMSprop(model.parameters(),
                                lr=init_lr,)
loss_f = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, 
                                                       mode='min', 
                                                       patience=5, 
                                                       factor=lr_factor, 
                                                       threshold=min_lr/lr_factor, # Dividimos entre el factor para usarlo en el early stopping
                                                       verbose=True)

# Iteramos cada época
for epoch in np.arange(n_epochs):
    # Iteramos sobre los datos
    train_loss = 0
    # Ponemos el modelo en modo train
    model.train()
    for x,y in  h5_generator(train_file, source, target, train_samples, batch_size, True):
        # Anulamos los gradientes
        optimizer.zero_grad()
        # Cargamos los datos del batch en GPU/dispositivo
        x = torch.tensor(x, dtype=torch.float32, device=device)
        y = torch.tensor(y, dtype=torch.float32, device=device)
        
        x = model.encoder_stft(x)
        y = model.encoder_stft(y)
        
#         x = encoder(x)
        # Alternativamente, podemos cortar la señal y a la longitud de y_pred, solo se pierden ~ 0.01 segundos de audio
#         y_decoded = model.decoder_stft(model.encoder_stft(y))
        
        # Hacemos el forward pass
        y_pred = model(x)
        
        # Calculamos el error entre el y_pred y el y_decoded
        loss = loss_f(y, y_pred)
        # Acumulamos para después
        train_loss+=loss.item()
        # Hacemos el back propagation (ahora los gradientes de cada parámetro se actualizan)
        loss.backward()
        
        # Actualizamos los parámetros haciendo un paso del optimizador
        optimizer.step()
        
        # Actualizamos el progreso de entrenamiento
        progress_bar.update(x.size()[0])
    
    val_loss = 0
    # Ponemos el modelo en modo test
    model.eval()
    with torch.no_grad():
        for x,y in h5_generator(val_file, source, target, val_samples, batch_size, False):
            x = torch.tensor(x, dtype=torch.float32, device=device)
            y = torch.tensor(y, dtype=torch.float32, device=device)

            x = model.encoder_stft(x)
            y = model.encoder_stft(y)

            y_pred = model(x)

            loss = loss_f(y,y_pred)
            val_loss += loss.item()
    
    print("Epoch %d" % (epoch+1))
    print("Train loss: %f / %f" % (train_loss, train_loss/train_samples))
    print("Validation loss: %f / %f" % (val_loss, val_loss/val_samples))
    print()
    
    # Ahora las estrategias de cambio de lr
    # Llamamos al scheduler de LR
    scheduler.step(val_loss)
    scheduler.verbose
    
    # Si el lr llega al mínimo, paramos el entrenamiento
    if(optimizer.state_dict()['param_groups'][0]['lr'] <= min_lr):
        # Paramos el entrenamiento
        print("Early stopping")
        break

  0%|          | 0/634700 [00:00<?, ?it/s]

Epoch 1
Train loss: 3690.813028 / 0.581505
Validation loss: 892.995320 / 0.727788

Epoch 2
Train loss: 3237.210498 / 0.510038
Validation loss: 861.193187 / 0.701869

Epoch 3
Train loss: 3093.264973 / 0.487359
Validation loss: 858.844032 / 0.699954

Epoch 4
Train loss: 2971.862754 / 0.468231
Validation loss: 925.808130 / 0.754530

Epoch 5
Train loss: 2933.945182 / 0.462257
Validation loss: 856.275753 / 0.697861

Epoch 6
Train loss: 2869.658204 / 0.452128
Validation loss: 890.140333 / 0.725461

Epoch 7
Train loss: 2834.135901 / 0.446532
Validation loss: 929.335000 / 0.757404

Epoch 8
Train loss: 2788.863888 / 0.439399
Validation loss: 863.219581 / 0.703520

Epoch 9
Train loss: 2777.979912 / 0.437684
Validation loss: 847.192729 / 0.690459

Epoch 10
Train loss: 2736.862098 / 0.431206
Validation loss: 893.023521 / 0.727811

Epoch 11
Train loss: 2703.613457 / 0.425967
Validation loss: 872.985177 / 0.711479

Epoch 12
Train loss: 2676.148630 / 0.421640
Validation loss: 824.169062 / 0.671694

E

In [13]:
# Guardamos el modelo
torch.save(model, data_path+('model_%s.pt'%(target)))

In [14]:
sample_idx = 0

In [15]:
sample = (model.decoder_stft(y, length=132300))
sample = sample[sample_idx].cpu().detach()
ipd.Audio(sample, rate=44100)

In [16]:
sample = (model.decoder_stft(y_pred, length=132300))
sample = sample[sample_idx].cpu().detach()
ipd.Audio(sample, rate=44100)

In [17]:
sample = (model.decoder_stft(x, length=132300))
sample = sample[sample_idx].cpu().detach()
ipd.Audio(sample, rate=44100)